In [1]:
import cv2
import numpy as np
from keras.preprocessing import image
import tensorflow as tf
import os

def nothing(x):
    pass

try: 
    os.mkdir('./temp') 
except: 
    pass 

image_x, image_y = 64,64

from keras.models import load_model
classifier = load_model('Trained_model.h5')

def predictor(img):
    cv2.imwrite(r".\temp\1.png",img)
    img = image.load_img(r".\temp\1.png",target_size=(64,64))

    test_image = image.img_to_array(img)
    #        print(test_image.shape)
    test_image = np.expand_dims(test_image, axis = 0)
    #        test_image = test_image.reshape((64,64,3))
    # predict image using classifier
    result = classifier.predict(test_image)

    if result[0][0] == 1:
          return 'A'
    elif result[0][1] == 1:
          return 'B'
    elif result[0][2] == 1:
          return 'C'
    elif result[0][3] == 1:
          return 'G'
    elif result[0][4] == 1:
          return 'I'
    elif result[0][5] == 1:
          return 'L'
    elif result[0][6] == 1:
          return 'O'
    elif result[0][7] == 1:
          return 'R'
    elif result[0][8] == 1:
          return 'W'
    elif result[0][9] == 1:
          return 'Y'
    


cam = cv2.VideoCapture(0)

cv2.namedWindow("Trackbars")

cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 123, 255, nothing)

# cv2.namedWindow("test")

img_counter = 0

img_text = ''
while True:
    ret, frame = cam.read()
    frame = cv2.flip(frame,1)
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")


    img = cv2.rectangle(frame, (425,100),(625,300), (0,255,0), thickness=2, lineType=8, shift=0)

    lower_blue = np.array([l_h, l_s, l_v])
    upper_blue = np.array([u_h, u_s, u_v])
    imcrop = img[102:298, 427:623]
#     imcrop = cv2.resize(imcrop,(64,64))
#     print(imcrop.shape)
    hsv = cv2.cvtColor(imcrop, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    cv2.putText(frame, img_text, (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0, 255, 0))
    cv2.imshow("test", frame)
    cv2.imshow("mask", mask)
    

    if cv2.waitKey(1)==ord('q'):
        break
        
    if(np.count_nonzero(mask)<=10000):
        continue

    if mask.any():
        img_text = predictor(mask)
    else:
        continue

    

cam.release()
cv2.destroyAllWindows()